In [1]:
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
rcParams.update({'font.size':16})


import numpy as np
from ipywidgets import widgets
import matplotlib.pyplot as plt

from mpmath import mp
mp.dps = 50

In [16]:

def spec(f,k,V,t):
    t_n = t*mp.exp(k*V/(mp.mpf(4.04*10**-21)))
    return mp.mpf((2 * t_n)/(mp.pi*(1+mp.power(2,t_n*f))))

def F1(f,b,T2):
    w_p = mp.mpf(2.865E9 + 28E6*b)
    w_n = mp.mpf(2.865E9 - 28E6*b)
    k = mp.mpf(1/T2)
    return mp.mpf((1/mp.pi)*( k/(k**2 + mp.power(f-w_p,2)) + k/(k**2 + mp.power(f-w_n,2))))

def F2(f,tau):
    return (1/(4*mp.pi**2)) * (2/tau) * mp.sin(2*mp.pi*tau*f/4.)**4 / (f/4.)**2

def T1_pred(b,r,b_v,k,t):
   
    V =  mp.mpf(4/3.0*np.pi*(r*1E-9)**3)
    w_p = mp.mpf(2.865E9 + 28E6*b)
    w_n = mp.mpf(2.865E9 - 28E6*b)
    i = lambda f: F1(f,b,T2)*spec(f,k,V,t)
    S = mp.quad(i,[1E8,w_n-1E8,w_n+1E8,w_p-1E8,w_p+1E8,1E10])
    G_ext = mp.mpf(28E6**2*b_v**2*S)
    return mp.mpf(1/(1/T1+G_ext))

def T2_pred(tau,r,b_v,k,t):
    V =  mp.mpf(4/3.0*np.pi*(r*1E-9)**3)
    i = lambda f: F2(f,tau*2)*spec(f,k,V,t)
    S = mp.quad(i,[1E4,1/tau-1E6,1/tau+1E6,1E8])
    G_ext = mp.mpf(28E6**2*b_v**2*S)
    return mp.mpf(1/(1/T2+G_ext))


In [11]:
def pred_time(T_pred,g):    
    return 1 / np.sum( g/np.sum(g)*(1/np.array(T_pred)) )

def gaussian(X,mu,sigma):
    return np.exp( -(X-mu)**2/(2*sigma**2) )

In [4]:
MNP2_T1 = np.array([1460,386,4012,195,525,2770,1258,1075,324,483,1795,15735,537,339,1319,2247,4462,2217,751,678])*1E-9
MNP11_T1 = np.array([21261,8448,35817,3809,6106,582])*1E-9
MNP5_T1 = np.array([6199,959,5998,5619])*1E-9
MNP12_T1 = np.array([27245,3128,4254,11391,14723,12519,42427,16878,47553,25542,39281])*1E-9
bare_T1 = np.array([292853,319588,100662,31148,54657,46828,99135,229618])*1E-9

MNP2_T2 = np.array([256,829])*1E-9
MNP4_T2 = np.array([926,1226,961,939,729,567,2194,433,1136])*1E-9
MNP5_T2 = np.array([124,654])*1E-9
MNP12_T2 = np.array([471,597,670])*1E-9
bare_T2 = 2074E-9

In [12]:
T1_size = [ [5.6,1.9], [19,4.5], [21,3.7], [2.53,0.76]]
T2_size = [ [5.6,1.9], [19,4.5], [2.53,0.76]]

In [13]:
T1s = list(np.log10(np.mean(T)) for T in [MNP2_T1,MNP5_T1,MNP11_T1,MNP12_T1])
T2s = list(np.log10(np.mean(T)) for T in [MNP2_T2,MNP5_T2,MNP12_T1])

In [ ]:
b_v = 2.8
R = np.arange(1,15,0.1)

T1 = np.mean(bare_T1)
T2 = np.mean(bare_T2)

k = np.linspace(1E4,5E4,30)
t = np.linspace(-12,-8,30)

tl1 = np.zeros((30,30,R.shape[0]))
tl2 = np.zeros((30,30,R.shape[0]))

for i,K in enumerate(k):
    for j,T in enumerate(t):
        
        T1_0 = list( T1_pred(0,r,b_v*(r/3),K,T) for r in np.nditer(R))
        T2_0 = list( T2_pred(5E-7,r,b_v*(r/3),K,T) for r in np.nditer(R))
        tl1[i,j,:] = T1_0
        tl2[i,j,:] = T2_0